# Exploratory data analysis,Anomaly Detection and ML predictions on Time Series energy and power datasests

In [ ]:
# Printed tables and result can't be shown on github since the all file is too large to render by Github.

In [ ]:


# Importing neccesary libs as initial
#İlk gerekli kütüphanelerin yüklenmesi 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
from collections import Counter

In [ ]:
#Loading the data
features =pd.read_csv ("features.csv", sep=',')
df = features.copy()

In [ ]:
#Initial xploratory data analysis steps
#Keşifçi ön analiz işlemleri
pd.set_option('display.max_columns', None)
df.head(5)

In [ ]:
# Let's see  data types
#Veri türlerini görelim
df.info ()

In [ ]:
# Here we see descriptive statistics
## Betimsel istatistikleri alalım
df.describe (include = 'all')

#### Date time variables should be converted to DateTime format in order to let run ML algorithms properly

#### "DateTime" değişkeninin ML algoritması için uygun tarih formatına çevrilmesi ve Object yerine DateTime formatında kaydedilmesi gerekir

In [ ]:
df['Timestamp'] =  pd.to_datetime(df['Timestamp'], infer_datetime_format=True)

In [ ]:
## Gaps in column title names have been filled properly 
#### Sütun başlıklarındaki boşluklar dolduruldu
df.columns = df.columns.str.replace(' ','_')
df.columns = df.columns.str.replace('-','_')

In [ ]:
#Here we have imported power.csv as well , we gona merge it later
#Power veri setini çektik
power=pd.read_csv ("power.csv", sep=',')
power.head(2)

In [ ]:
power.info()

In [ ]:
# by using df.describe() method, "-273" values for first three numeric columns have been placed as minimum values.
# so it is better to call min values for all columns in order to see if there is sytematic anomaly for all columns 

# df. describe() ile sütunlarda min değerlere baktığımızda -273'lerin ilk 3 nümerik değişkende min değer olduğunu gördük
#burdan hareketle daha sonra lazım olabilir diye min value değerleri ayrıyetten çekmek istedik
minvalues = df.min()
minvalues

In [ ]:
# Here, IQR statistical method have been implemented to detect  all anormalies for all columns. And all printed values have been saved as series.
#  Veri seti içindeki tüm nümerik değişkenlerin hepsini gezerek anomali tespiti yapıp outlier olarak kaydedecek formülü yazdık. Ve series kaydettik

def find_outliers(col):
    q1 = col.quantile(.25)
    q3 = col.quantile(.75)
    IQR = q3 - q1
    ll = q1 - (1.5*IQR)
    ul = q3 + (1.5*IQR)
    upper_outliers = col[col > ul].tolist()
    lower_outliers = col[col < ll].tolist()
    bad_indices = list(set(upper_outliers + lower_outliers))
    return(bad_indices)

import numpy as np
outliers = []
for col in df.columns:
    if df[col].dtype in ["int64","float64"]:
        outliers.append(find_outliers(df[col]))
        

print(len(outliers))

In [ ]:
# Lets check firt numeric columns by of outliers calling[0] . All values are saved as pandas.core.series
# ilk sıradaki[0] nümerik değişken sütünu için kaydedilen outlier değerleri çektik. Değerler pandas.core.series. olarak kaydedilmiş
outliers[0]

In [ ]:
# Here we save all outlie values as a Pd.dataframe
# Outlier değerleri Pandas dataframe olarak kaydedelim
dff= pd.DataFrame(outliers)
dff .head ()

In [ ]:
# These values can be converted to a numpy array and saved as an array of unique values.
# Bu değerleri numpy array olarak çevirip uniqe değerler olarak alalım ki tekrar etmesinler 
arr = dff.to_numpy()
uniq =np.unique(arr)

In [ ]:
#Now we save at as list and here we control the length of the list
uniq =uniq.tolist()
len(uniq)

In [1]:
#while creating the outliers list by implementing the IQR method, the "0" values have been accidently chosen as an outlier although its not an outlier
# here we need to delete "0" from outliers list 

# outlier seçimi esnasında bir hata olarak "0" değerinin de outlier olarak kaydedildiğini gördük
# bu hatadan kurtulmak için "0" değerini listeden çıkaracağız. 

In [ ]:
0 in uniq


In [ ]:
uniq.remove(0)
0 in uniq

In [ ]:
# Now we can replace all outlier values with np.nan
#since it takes a lot time , I have already saved np.nan replaced dataframe before while i had been processing the first code line as below
# now we can continue with the already processed sonout.csv

# Aşağıdaki formül ile Features veri seti içindeki önceden tespit edilen yani outliers olan tüm değerleri sileceğiz ve np.nan yapacağız
# süreç çok sürdüğü için bu işlemi önceden yapmış ve kaydetmiştik. Ara verdikten sonra kaldığımız yerden devam etmek için
# önceden kaydettiğimiz sonout.csv üzerinden işlemlere devam ediyoruz.
df = df.replace([uniq],np.nan)
df.to_csv('sonout.csv')


In [ ]:
df=pd.read_csv ("sonout.csv", sep=',')


## Filling işlemleri
### We could fill np.nan values with np.mean and np.median values calculated on daily basis. Since hourly recorded values differ a  lot during the day , it would ruin Central Tendency of whole dataframe. Thus, forwardfill and backfill methods have been implemented.


## Filling işlemleri

### Boşlukları günlük bazda hesaplanan mean ya da median değerler olarak doldurabilirdik . Ancak bu yöntem gün içi saatlik bazda veriler çok değişkenlik arzettiği için verinin merkezi eğiliminin bozulmasına neden oluyor. Bundan dolayı önce forwardfill sonra backfill yaptık.

In [ ]:
import numpy as np
names = df.columns[(df.dtypes.values == np.dtype('float64'))]
type(names)

names = names.unique()
liste= list(names)

In [ ]:

for col in names:
    df[col] = df[col].ffill()
    
for col in names:
    df[col] = df[col].bfill()

In [ ]:
# Here we check if we still have null values
# Eksik değerlerimiz kaldı mı diye kontrol ettik
df.isnull().sum()

In [ ]:
# Here we have imported power.csv again
#Power.csv veri setini yendien çekelim
power=pd.read_csv ("power.csv", sep=',')
power.head(2)

## Exploratory data analysis
#### By using the exploratory data analysis, we can see the relationships between different variables. If a correlation coefficient is significant we can consider to use it for ML operations

## Keşifçi Veri Analizi İşlemleri

#### Koreleasyon Katsayılarını kullanarak değişkenler arasındaki ilişkiyi incelemek istiyoruz.Anlamlı ilişki bulunursa ML Modeli üzerinde kullanabiliriz.

In [ ]:
# Let's convert the object type TimeStamp to date time variable
# object olan TimeStamp'ı datetime olarak çevirelim.
power['Timestamp'] =  pd.to_datetime(power['Timestamp'], infer_datetime_format=True)
df['Timestamp'] =     pd.to_datetime(df['Timestamp'], infer_datetime_format=True)

In [ ]:
# Lets creat a train df,first we need to merge power to df
# train etmek üzere yeni dataframe oluşturalım, önce merge işlemi yapmamız gerekiyor.Features ve power birleştirdik
dftrain =pd.merge (df,power, on = "Timestamp")

In [ ]:
# but first , it is better to see a correlation matrix on unmerged , df dataframe
# ancak önce eski df veri seti üzerinden correlation matrix oluşturulalım,
corr_matrix = df.corr().abs()
corr_matrix

In [ ]:
# the varibles having very high correlation coefficient values may cause overfitting problem
# so we need to have variables with cor values between  0.5 and 0.85 

# Koreleasyon değerleri 0.85'den üstün olan değerleri almak yanıltıcı olur, overfitting sorunu ortaya çıkartabilir
# bunun için en 0.5 ve 0.85 arasındaki koreleasyon matrislerini baz alacağız. 

# Here we extract uppertriangular matrix 
# Üst üçgen correlation matrix seçelim
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

# here we remove less than 0.50 corr values
#  0.50 üzeri corr matrix çıkaran değikenleri seçelim
correlated = [column for column in upper.columns if any(upper[column] > 0.5)]

#Here we remove more than 0.85 corr values
# 0.85 üzeri corr matrix çıkaran değikenleri eleyelim
to_drop = [column for column in upper.columns if any(upper[column] > 0.85)]

In [ ]:
# Here we remove to_drop list we created as above from the df
# elenenleri silelim
df.drop (to_drop, axis=1, inplace=True)

In [ ]:
# Now we can see best correla variables
# en uygun corr matrix içeren değişkenleri aldık
df.corr().unstack().drop_duplicates().sort_values().tail(15)

In [ ]:
#Lets merge it again after dropped variables
# elenen değişkenlerden sonra yeniden  merge yaparak dataframe oluşturalım
dftrain =pd.merge (df,power, on = "Timestamp")

## Creating a Correalation Matrix of Independent Variables explaning the Power dependent variable the best 

## POWER hedef değişkenini en iyi açıklayan nümerik bağımsız değişkenleri listelemek için yeniden bir corr matrix alalım¶

In [ ]:
dftrain.corr().unstack().drop_duplicates().sort_values().tail(30)

In [ ]:
# As an example, here we created a plot table of "Converter_Control_Unit_Reactive_Power"
# Örnek olarak "Converter_Control_Unit_Reactive_Power" seçerek bir plot tablo aldık
df.plot.line(y="Converter_Control_Unit_Reactive_Power", use_index=True).figure

In [ ]:
# in or do to understand it better here check descriptive statisctic of df train
# yeni veri setimizin son durumuna bir bakmak üzere betimseli istatistik alalım
dftrain.describe().T

##  Now we can split test and train datas

## ML Modeli için test train seçimine başlayalım

In [ ]:
# Dependent and independent variables are chosen
#  Bağımsız ve bağımlı değişken ayrımları yapıldı
X = dftrain.iloc[:,1:-1].values
y = dftrain.iloc[:, -1].values

In [ ]:
###  "from sklearn.model_selection import TimeSeriesSplit"

## as the code written above is TimeSeriesSplit.But results are not good enough on kaggle
### yukarıdaki gibi time series split de yapılabilirdi ancak denendi ve kaggle üzerinde fayda sağlamadı

In [ ]:
# Here we standartscale the inpdepentdent X variables 
# bağımsız X değişkenleri için standart scaling yapalım 


from sklearn.preprocessing import StandardScaler
scalerX = StandardScaler().fit(X_train)
X_train= scalerX.transform(X_train)
X_test= scalerX.transform(X_test)

### Ont this dataset, predictions are made with LightGBM
### Bu veri setinde LightGBM ile tahmin oluşturduk.



In [ ]:
! pip install lightgbm
import lightgbm as ltb

model = ltb.LGBMRegressor()
model.fit(X_train, y_train)
print(model)

## As a result of predictions; test data achieved 0.93 rmse score on train data
### I had also implemented XGBRegressor model and RandomizedSearchCV tuning however reuslts were not better on kaggle. It may work better with another dataset



### Yukarıdaki train veri seti üzerinde test için ayrılan veri setimiz 0.93 score elde etti.
###  Aslında önceden XGBRegressor modeli ve RandomizedSearchCV yöntemiye tuning yaparak tahminde bulunmuştuk ancak kaggle üzerinde score daha düşük geldi.

# Now we create a modelling prediction for desired unknown future values. 

## Şimdi modelleme tahmini yapmak üzere  prediction veri seti oluşturalım.

In [ ]:
#Here we create the prediction dataset

# prediction dataset oluşturalım.
dfpred = pd.concat([df,power],axis =1)
len(dfpred)
dfpred=dfpred[["Gearbox_T1_High_Speed_Shaft_Temperature", "Tower_Acceleration_Normal","Temperature_Trafo_3", "Temperature_Bearing_A",
"Converter_Control_Unit_Reactive_Power","Temperature_Shaft_Bearing_1","Moment_D_Filtered", "Power(kW)"  ]]
dfpred = dfpred[dfpred["Power(kW)"].isnull()]

In [ ]:
# We need to choose datas for x_predict and run the model
# X_predict üzerinde tahminde kullanılacak verileri seçtik ve modelimizi çalıştırdık.
X_predict = dfpred.iloc[:,:-1].values


predicted_y = model.predict(X_predict)

In [ ]:
#Let's see predictions
predicted_y

## Our predictions are ready, lets create the submission dataset

### Yukarıda tahminlerimiz geldi, artık submission.csv veri setimizi oluşturabiliriz.

In [ ]:
powerarray= pd.DataFrame(predicted_y, columns = ['Power(kW)'])
powerarray.head(2)

In [ ]:
sample= pd.read_csv ("sample_submission.csv", sep=',')
sample= pd.DataFrame().assign(Timestamp=sample['Timestamp'])
sample.head(2)

In [ ]:
samplesubmission =pd.merge (sample,powerarray, left_index=True,right_index=True)
samplesubmission .head ()